In [ ]:
"""
@author: Nushrat
"""

In [ ]:
get_ipython().magic('pylab inline')


In [ ]:
import datetime
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
import keras

In [ ]:
#TodasEstaciones = pd.read_csv('merged.csv',skipinitialspace=True, comment="#", sep=',', index_col=0, parse_dates=True)
TodasEstaciones = pd.read_csv('merged_new.csv',skipinitialspace=True, comment="#", sep=';', index_col=0, parse_dates=True)
TodasEstaciones

In [ ]:
#selected_TodasEstaciones= TodasEstaciones.loc['2017-01-01':'2018-01-01',['Value_y.1','Value_y.2','y1','x1','x2']]
selected_TodasEstaciones= TodasEstaciones.loc['1968-05-01':'1972-10-31',['Value_x','Value_y','Value_x.1','Value_y.1','Value']]
selected_TodasEstaciones

In [ ]:
#selected_TodasEstaciones['date']=selected_TodasEstaciones.index
dates = selected_TodasEstaciones.index.values.tolist()
#print(dates)
selected_TodasEstaciones.head()

In [ ]:
import datetime
#we create a date column to extract the week number
TodasEstaciones['date']=TodasEstaciones.index
TodasEstaciones.head()
for col in TodasEstaciones.columns: 
    print(col)

In [ ]:
########################################LSTM code starts#######################

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(selected_TodasEstaciones.isnull())
plt.show()



In [ ]:
############trying LSTM
from sklearn.preprocessing import MinMaxScaler

selected_TodasEstaciones.dropna(inplace=True)
#print(selected_TodasEstaciones.head())
dataset = selected_TodasEstaciones.copy()
dataset.reset_index(inplace=True, drop=True)
scaler = MinMaxScaler()

#reshape  = ['Value_y.1','Value_y.2','y1','x1','x2']
reshape = ['Value_x','Value_y','Value_x.1','Value_y.1','Value']
dataset[reshape] = scaler.fit_transform(dataset[reshape])
#dataset = scaler.fit_transform(dataset)

In [ ]:
dataset

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test =train_test_split(dataset , train_size = 0.75 , random_state = 38)

In [ ]:
rate = 0.75  
tst_train_len = int(len(dataset)*rate)
tst_test_len = len(dataset) - tst_train_len
print(tst_train_len)
train, test = dataset.loc[0:tst_train_len,:],dataset.loc[tst_train_len:len(dataset),:]
#dataset.loc[30317,:]
#print(range(len(test)-look_back-1))
#test[1,:]

In [ ]:
dataset.index

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    print(range(len(dataset)-look_back-1))
    for i in range(len(dataset)-look_back-1):
        a = np.array(dataset.loc[i:(i+look_back), :])        
        dataX.append(a)
        #print(i+look_back)
        #dataset.loc[i + look_back, :]
        dataY.append(np.array(dataset.loc[i + look_back, :]))
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_dataset2(dataset, look_back=1, ofs=0):
    dataX, dataY = [], []
    for i in range(ofs,len(dataset)-look_back-1+ofs):
        a = np.array(dataset.loc[i:(i+look_back), :])                  
        dataX.append(a)
        dataY.append(np.array(dataset.loc[i + look_back , :]))
    return np.array(dataX), np.array(dataY)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import ModelCheckpoint
look_back = 3
trainX, trainY = create_dataset(train,look_back)

#testX, testY = create_dataset(test,look_back)

In [ ]:
print(len(test))
test.reset_index(inplace=True,drop=True)
print(test.index)
#test.loc[0,:]

In [ ]:
testX, testY = create_dataset(test,look_back)
#testX, testY = create_dataset2(test,look_back,1108)

In [ ]:
cnt_perc = 4
cnt_epoch = 300
bz = 1
model = Sequential()
model.add(LSTM(cnt_perc, input_shape=(look_back+1,5)))
model.add(Dense(5))
model.summary()
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=opt) # put learning rate
model.fit(trainX, trainY, epochs=cnt_epoch, batch_size=bz)
trainPredict = scaler.inverse_transform(model.predict(trainX))
testPredict = scaler.inverse_transform(model.predict(testX))

In [ ]:
dataset = selected_TodasEstaciones.copy()
dataset.reset_index(inplace=True, drop=True)
scaler = MinMaxScaler()

reshape  = ['Value_x','Value_y','Value_x.1','Value_y.1','Value']

dataset[reshape] = scaler.fit_transform(dataset[reshape])
original = scaler.inverse_transform(dataset)
print (original[0:10,:])
selected_TodasEstaciones.head(10)

In [ ]:
trainPredict = np.round(scaler.inverse_transform(model.predict(trainX)))
testPredict = scaler.inverse_transform(model.predict(testX))

In [ ]:
print (len(original))
print (len(trainX))
print (len(trainX)+(look_back*2)+1)

In [ ]:
from sklearn.metrics import mean_squared_error
import math

trainScore = math.sqrt(mean_squared_error(original[look_back:(len(trainX)+look_back),4], trainPredict[:,4]))
print('Train: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(original[len(trainX)+(look_back*2)+1:,4], testPredict[:,4]))
print('Test: %.2f RMSE' % (testScore))

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(original[:,4],label='original',color='red')
plt.plot(trainPredict[:,4],label='prediction',color='blue')
#plt.plot(testPredict[:,4])
plt.show()
plt.savefig('NorthAmerica_prediction_train_LSTM.png')

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(original[:,3],color='red')
plt.plot(testPredict[:,3],color='blue')
plt.show()

In [ ]:
start_date = pd.to_datetime(dates[0], format="%Y-%m-%d")
end_date = pd.to_datetime(dates[len(dates)-2], format="%Y-%m-%d")
start_date = start_date
end_date = end_date + pd.DateOffset(days=1)
date_range = pd.date_range(start_date, end_date)
print(date_range.shape)
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(date_range[:1636],original[:,4], label="observation",color='red')
ax.plot(date_range[:1224], trainPredict[:,4], label="prediction",color='blue')
ax.legend()
ax.set_title(f"Filling missing values from 1968 to 1973")
ax.xaxis.set_tick_params(rotation=90)
ax.set_xlabel("Date")
_ = ax.set_ylabel("Discharge (mm/d)")
plt.show()


In [ ]:
all_methods_2.head()

In [ ]:
dates = all_methods_2.index.values.tolist()

In [ ]:
observation = all_methods_2.loc[:,'Observation']
prediction =all_methods_2.loc[:,['Interpolation','Kalman','Locf','Mean','MovingAverage']]

In [ ]:
prediction.head()

In [ ]:
start_date = pd.to_datetime(dates[0], format="%Y-%m-%d")
end_date = pd.to_datetime(dates[len(dates)-2], format="%Y-%m-%d")
start_date = start_date
end_date = end_date + pd.DateOffset(days=1)
date_range = pd.date_range(start_date, end_date)
print(date_range.shape)
fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(date_range,observation, label="observation",color='red')

#ax.plot(date_range, prediction['Interpolation'], label="Interpolation",color='blue',linestyle='dashed')
#ax.plot(date_range, prediction['Kalman'], label="Kalman",color='green',linestyle='dashed')
#ax.plot(date_range, prediction['Locf'], label="Locf",color='purple',linestyle='dashed')
#ax.plot(date_range, prediction['MovingAverage'], label="MovingAverage",color='black',linestyle='dashed')
ax.plot(date_range, prediction['Mean'], label="Mean",color='gray',linestyle='dashed')

ax.legend()
ax.set_title(f"Filling missing values from 1968 to 1973")
ax.xaxis.set_tick_params(rotation=90)
ax.set_xlabel("Date")
_ = ax.set_ylabel("Discharge (mm/d)")
plt.show()
